# <font size="6"> Predict Whether A Candidate is A Real Pulsar

# <font size="5"> 1.Introduction

**Pulsars** are rare neutron stars, as probes of space-time, the interstellar medium, and the state of matter, which are very important for the study and development of natural sciences(Wikipedia). The search for pulsars relies mainly on detecting signals emitted by periodic broadband radio emission patterns (averaged over multiple rotations) as they rotate at high speeds. However, in practice, all detections are caused by radio frequency interference (RFI) and noise, so it is difficult to find legitimate signals. Hence, the search for **real** pulsars is challenging. 

**Our question is could we accurately predict a real pulsar using predictor variables Mean_IP, ExcessKurtosis_IP, and ExcessKurtosis_DS?**

HTRU2 is a dataset on *kaggle* describing a sample of pulsar candidates collected during the High Temporal Resolution Universe Survey (South). It contains 16,259 spurious examples caused by RFI/noise and 1,639 real pulsar examples. Eight continuous variables describe each candidate as below and first four of them are statistics obtained from integrated pulse profile and the remaining are from DM-SNR curve.This is an array of continuous variables that describe a longitude-resolved version of the signal that has been averaged in both time and frequency.Please see the information as below,
* Mean of the integrated profile.
* Standard deviation of the integrated profile.
* Excess kurtosis of the integrated profile.
* Skewness of the integrated profile.
* Mean of the DM-SNR curve.
* Standard deviation of the DM-SNR curve.
* Excess kurtosis of the DM-SNR curve.
* Skewness of the DM-SNR curve.
* Class:1 means real pulsar and 0 otherwise.

We will further explain the details on the integrated profile and the DM-SNR curve respectively in the following report.

**Project Model Selection**: We choose classification algorithms - K nearest neighbors algorithms for our project. As we would like to predict the "Class" of a candidate and "Class" is a categorical class.

# <font size="5"> 2.Methods and Results

First of all, we need to import all of the Jupyter libraries we are going to apply for this project.
Load the `tidyverse`, `tidymodels` and `GGally` package for data analysis.

In [1]:
library(tidyverse)
library(tidymodels)      
install.packages("GGally")                    
library("GGally")
library(rvest)
install.packages("vcd")
library(vcd)
library(ggplot2)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

Warning message:
“package ‘ggplot2’ was built under R version 4.0.1”
Warning message:
“package ‘tibble’ was built under R version 4.0.2”
Warning message:
“package ‘tidyr’ was built under R version 4.0.2”
Warning message:
“package ‘dplyr’ was built under R version 4.0.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Warning message:
“package ‘tidymodels’ was built under R version 4.0.2”
── Attaching packages ────────────────────────────────────── tidymodels 0.1.1 ──

✔ broom     0.7.0      ✔ recipes   0.1.13
✔ dials     0.0.9      ✔ rsample   0.0.7 
✔ infer     0.5.4      ✔ tune      0.1.1 
✔ modeldata 0.0.2      ✔ workflows 0.2.0 
✔ parsnip   0.1.3    

Set the seed only once before loading data to gurantee our analysis to be reproducible.

In [2]:
set.seed(1)

Read the file from online using `read_table2`. Load the data and add column names using `rename` manually to clean and wrangle data into tidy format. Also factor our predictor for classification using the `as_factor` function. 

In [4]:
pulsar_raw_data <- read_table2("https://raw.githubusercontent.com/JeanetteOfficial/ActiveDSCI100_Group_006-1_proj/main/data/HTRU_2.txt",
                          col_names = FALSE) 
pulsar_raw_data <- rename(pulsar_raw_data, "Mean_IP" = X1,
                         "SD_IP" = X2,
                         "ExcessKurtosis_IP" = X3,
                         "Skewness_IP" = X4,
                         "Mean_DS" = X5,
                         "SD_DS" = X6,
                         "ExcessKurtosis_DS" = X7,
                         "Skewness_DS" = X8,
                         "Class" = X9) 

pulsar_raw_data <- mutate(pulsar_raw_data, Class = as_factor(case_when(Class == 1 ~ "pulsar",
                                                                      TRUE ~ "rfi_noise")))
head(pulsar_raw_data)

Parsed with column specification:
cols(
  X1 = col_double(),
  X2 = col_double(),
  X3 = col_double(),
  X4 = col_double(),
  X5 = col_double(),
  X6 = col_double(),
  X7 = col_double(),
  X8 = col_double(),
  X9 = col_double()
)



Mean_IP,SD_IP,ExcessKurtosis_IP,Skewness_IP,Mean_DS,SD_DS,ExcessKurtosis_DS,Skewness_DS,Class
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
140.56250,55.68378,-0.23457141,-0.6996484,3.199833,19.11043,7.975532,74.24222,rfi_noise
102.50781,58.88243,0.46531815,-0.5150879,1.677258,14.86015,10.576487,127.39358,rfi_noise
103.01562,39.34165,0.32332837,1.0511644,3.121237,21.74467,7.735822,63.17191,rfi_noise
136.75000,57.17845,-0.06841464,-0.6362384,3.642977,20.95928,6.896499,53.59366,rfi_noise
88.72656,40.67223,0.60086608,1.1234917,1.178930,11.46872,14.269573,252.56731,rfi_noise
93.57031,46.69811,0.53190485,0.4167211,1.636288,14.54507,10.621748,131.39400,rfi_noise


**Table 1**: Preview the raw data after factoring the Class.

We use  `glimpse` to inspect the data, as we have a lot of columns. In such way, it prints the data such that the columns go down the page (instead of across). And we can easily view our 9 variables.

In [10]:
glimpse(pulsar_raw_data)

Rows: 17,898
Columns: 9
$ Mean_IP           <dbl> 140.56250, 102.50781, 103.01562, 136.75000, 88.7265…
$ SD_IP             <dbl> 55.68378, 58.88243, 39.34165, 57.17845, 40.67223, 4…
$ ExcessKurtosis_IP <dbl> -0.23457141, 0.46531815, 0.32332837, -0.06841464, 0…
$ Skewness_IP       <dbl> -0.69964840, -0.51508791, 1.05116443, -0.63623837, …
$ Mean_DS           <dbl> 3.1998328, 1.6772575, 3.1212375, 3.6429766, 1.17892…
$ SD_DS             <dbl> 19.110426, 14.860146, 21.744669, 20.959280, 11.4687…
$ ExcessKurtosis_DS <dbl> 7.975532, 10.576487, 7.735822, 6.896499, 14.269573,…
$ Skewness_DS       <dbl> 74.24222, 127.39358, 63.17191, 53.59366, 252.56731,…
$ Class             <fct> rfi_noise, rfi_noise, rfi_noise, rfi_noise, rfi_noi…


**Table 2**: Preview the raw data with column names can be read from up to down.

Explore the data using  `group_by`,  `summarize`,  `n` functions within  `summarize` to find the number and percentage of pulsar and rfi_noise observations in our data set. We observed that it has 1639 (9%) pulsar and 16259 (91%) rfi_noise observations. It is predicted as that there should be rare real pulsar.

In [11]:
pulsar_proportions <- pulsar_raw_data %>%
                      group_by(Class) %>%
                      summarize(n=n()) %>%
                      mutate(percent = 100*n/nrow(pulsar_raw_data))
pulsar_proportions

`summarise()` ungrouping output (override with `.groups` argument)



Class,n,percent
<fct>,<int>,<dbl>
rfi_noise,16259,90.842552
pulsar,1639,9.157448


**Table 3** : Percentage of Class 0 and Class 1 in the raw data.

Using  `initial_split ` split the train/test data for training an accurate model and achieving accurate evaluation for the model. As we experimented with different splits, we found that when the ratio of the training and testing data is 75% : 25%, it would be the most appropriate solution. Furthermore, according to the textbook, we conclude that the trade off between a larger training set making the model more accurate versus a larger testing set making our evaluation more accurate. As we experimented with different splits, we came to a conclusion that the 75-25 split would be the most appropriate solution. Also the `initial_split` function allows us to shuffle and stratify the data to prevent the order from influencing the outcome and to ensure that the proportion of each class is preserved between the training and testing split. Next, we use `glimpse` to inspect the training data. 

In [12]:
pulsar_split <- initial_split(pulsar_raw_data,prop=0.75,strata = Class)
pulsar_train <- training(pulsar_split)
pulsar_test <- testing(pulsar_split)
glimpse(pulsar_train)

Rows: 13,424
Columns: 9
$ Mean_IP           <dbl> 140.56250, 102.50781, 103.01562, 136.75000, 88.7265…
$ SD_IP             <dbl> 55.68378, 58.88243, 39.34165, 57.17845, 40.67223, 4…
$ ExcessKurtosis_IP <dbl> -0.234571412, 0.465318154, 0.323328365, -0.06841463…
$ Skewness_IP       <dbl> -0.69964840, -0.51508791, 1.05116443, -0.63623837, …
$ Mean_DS           <dbl> 3.199833, 1.677258, 3.121237, 3.642977, 1.178930, 1…
$ SD_DS             <dbl> 19.11043, 14.86015, 21.74467, 20.95928, 11.46872, 1…
$ ExcessKurtosis_DS <dbl> 7.975532, 10.576487, 7.735822, 6.896499, 14.269573,…
$ Skewness_DS       <dbl> 74.24222, 127.39358, 63.17191, 53.59366, 252.56731,…
$ Class             <fct> rfi_noise, rfi_noise, rfi_noise, rfi_noise, rfi_noi…


**Table 4**: Preview the training data.

We use `group_by`,  `summarize` and  `n` function within  `summarize` to find the perentange of Class  1 and Class 0 in `pulsar_train`. And we see about 1291(9%) are real pulsar and 12183 (91%) are rfi_noise, indicating that our class proportions were roughly preserved when we split the data, which means that we have enough data in each proportion in the training data. This also remind us that we need to frist balance the test data to use them.

In [13]:
pulsar_train_proportions <- pulsar_train %>%
                      group_by(Class) %>%
                      summarize(n=n()) %>%
                      mutate(percent = 100*n/nrow(pulsar_train))
pulsar_train_proportions

`summarise()` ungrouping output (override with `.groups` argument)



Class,n,percent
<fct>,<int>,<dbl>
rfi_noise,12183,90.755364
pulsar,1241,9.244636


**Table 5**: Percentage of Class 1 and Class 0 in the training data.

Check if there are missing values in the training data with the function `na.omit` . If so, we have to deal with them first, otherwise our data analysis would not be as accurate as possible. We can observe from above that the row count remains the same as the 75% of the raw data, meaning that there aren't any missing values. Check to see the first six rows of the data with `head`.

In [14]:
na.omit(pulsar_train, cols=c("Mean_IP", "SD_IP", "ExcessKurtosis_IP", "Skewness_IP", 
                             "Mean_DS", "SD_DS", "ExcessKurtosis_DS", "Skewness_DS","Class")) %>%
head(6)
# glimpse(pulsar_train)

Mean_IP,SD_IP,ExcessKurtosis_IP,Skewness_IP,Mean_DS,SD_DS,ExcessKurtosis_DS,Skewness_DS,Class
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
140.56250,55.68378,-0.23457141,-0.6996484,3.199833,19.11043,7.975532,74.24222,rfi_noise
102.50781,58.88243,0.46531815,-0.5150879,1.677258,14.86015,10.576487,127.39358,rfi_noise
103.01562,39.34165,0.32332837,1.0511644,3.121237,21.74467,7.735822,63.17191,rfi_noise
136.75000,57.17845,-0.06841464,-0.6362384,3.642977,20.95928,6.896499,53.59366,rfi_noise
88.72656,40.67223,0.60086608,1.1234917,1.178930,11.46872,14.269573,252.56731,rfi_noise
93.57031,46.69811,0.53190485,0.4167211,1.636288,14.54507,10.621748,131.39400,rfi_noise


**Table 6**: Preview the data after checking missing values.

`summarize` the means of the 8 predictor variables in a table so that we are able to know whether each data is higher or lower than the average.

In [15]:
mean_pulsar_train <- pulsar_train %>%
summarize(AVG_MeanIP = mean(Mean_IP, na.rm = TRUE),
          AVG_SDIP = mean(SD_IP,na.rm = TRUE),
          AVG_ExcessKurtosisIP = mean(ExcessKurtosis_IP,na.rm = TRUE),
          AVG_SkewnessIP = mean(Skewness_IP,na.rm = TRUE),
          AVG_MeanDS = mean(Mean_DS,na.rm = TRUE),
          AVG_SDDS = mean(SD_DS,na.rm = TRUE),
          AVG_ExcessKurtosisDS = mean(ExcessKurtosis_DS,na.rm = TRUE),
          AVG_SkewnessDS = mean(Skewness_DS,na.rm = TRUE))         
mean_pulsar_train

AVG_MeanIP,AVG_SDIP,AVG_ExcessKurtosisIP,AVG_SkewnessIP,AVG_MeanDS,AVG_SDDS,AVG_ExcessKurtosisDS,AVG_SkewnessDS
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
110.9139,46.5429,0.4840392,1.79401,12.69251,26.34225,8.314097,105.3613


**Table 7**: The means of all variables.

# <font size="4">Choosing Predictive Variables

**Intuition**: 

* Except for the Class variable, we have 8 continuous variables describing each candidate. First four are statistics obtained from integrated pulse profile. The pulse profile refers to the curve of the periodic variation of the pulsar's radiation signal with time, which is also known as the light curve. A single pulse signal is very unstable, the intensity of the pulse is variable, and sometimes it even disappears. However, the average value of thousands of pulses has a very stable periodicity(corr_ee). Therefore, we consider that the mean of the integrated pulse profile should be useful to judge and predict a real pulsar. 

* The remaining variables are from the DM-SNR(Dispersion Measure-Signal to Noise Ratio) curve. The DM-SNR curve records the relationship of SNR to DM, and the dispersion curve shows the corresponding SNR of the pulsed curve when different dispersion values are used for dedispersion. In the case of a pulsed signal, the peak of the curve appears at a non-zero position, while the curve for a non-pulse signal will either have no apparent peak or it peaks at zero(Liu. et al,12). Therefore, the Excess Kurtosis of the DM-SNR curve is essential for judging and predicting the pulsar.


Using `ggpairs` to compare the distribution of 8 variables in our dataset and to check with our intuition above. It seems that the mean of the Integrated pulsar profile and the Excess Kurtosis are well correlated to pulsar. However, the other six variables also work well. So we want to use histogram to check again.